In [343]:
print("Hello, we are 重磅赢家") # WE DID IT MAX!!! WE ARE IN KAGGLE!!!

Hello, we are 重磅赢家


In [344]:
# this is for https://www.kaggle.com/competitions/playground-series-s4e11/overview
import random
import numpy as np
import re
import pandas as pd # 貓熊
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import CategoricalNB
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [345]:
# load the data
train_data = pd.read_csv('train.csv')
train_data['Train'] = True
test_data = pd.read_csv('test.csv')
test_data['Train'] = False
df = pd.concat([train_data, test_data], axis=0).reset_index(drop=True)

In [346]:
df.head()

,id,Name,Gender,Age,City,Working Professional or Student,Profession,Academic Pressure,Work Pressure,CGPA,...,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression,Train
0,0,Aaradhya,Female,49.0,Ludhiana,Working Professional,Chef,NaN,5.0,NaN,...,2.0,More than 8 hours,Healthy,BHM,No,1.0,2.0,No,0.0,True
1,1,Vivan,Male,26.0,Varanasi,Working Professional,Teacher,NaN,4.0,NaN,...,3.0,Less than 5 hours,Unhealthy,LLB,Yes,7.0,3.0,No,1.0,True
2,2,Yuvraj,Male,33.0,Visakhapatnam,Student,NaN,5.0,NaN,8.97,...,NaN,5-6 hours,Healthy,B.Pharm,Yes,3.0,1.0,No,1.0,True
3,3,Yuvraj,Male,22.0,Mumbai,Working Professional,Teacher,NaN,5.0,NaN,...,1.0,Less than 5 hours,Moderate,BBA,Yes,10.0,1.0,Yes,1.0,True
4,4,Rhea,Female,30.0,Kanpur,Working Professional,Business Analyst,NaN,1.0,NaN,...,1.0,5-6 hours,Unhealthy,BBA,Yes,9.0,4.0,Yes,0.0,True


In [347]:
def parse_sleep_duration(duration_str):
    if isinstance(duration_str, str):
        duration_str = duration_str.lower()
        # Handle ranges like '6-8'
        if '-' in duration_str:
            match = re.search(r"(\d+)\s*-\s*(\d+)", duration_str)
            if match:
                a = int(match.group(1))
                b = int(match.group(2))
                avg = (a + b) / 2.0
                # Adjust if values seem to be weekly totals
                if a > 20 or b > 20:
                    avg /= 7
                return avg
        # Handle 'less than X' cases
        elif 'less than' in duration_str:
            match = re.search(r"less than\s+(\d+)", duration_str)
            if match:
                a = int(match.group(1))
                return random.uniform(1, a - 1)
        # Handle 'more than X' cases
        elif 'more than' in duration_str:
            match = re.search(r"more than\s+(\d+)", duration_str)
            if match:
                a = int(match.group(1))
                return random.uniform(a + 1, 12)  # Assume max 12 hours
        # Handle exact numbers
        else:
            match = re.search(r"(\d+)", duration_str)
            if match:
                return float(match.group(1))
    # Return NaN for unparseable entries
    return np.nan

# Apply the function to the 'Sleep Duration' column
df['Sleep Duration Numeric'] = df['Sleep Duration'].apply(parse_sleep_duration)
df['Sleep Duration Numeric'].fillna(df['Sleep Duration Numeric'].mean(), inplace=True)
df.drop(columns='Sleep Duration', inplace=True)
df.rename(columns={'Sleep Duration Numeric': 'Sleep Duration'}, inplace=True)

In [348]:
df.loc[df['Train']==False]


,id,Name,Gender,Age,City,Working Professional or Student,Profession,Academic Pressure,Work Pressure,CGPA,...,Job Satisfaction,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression,Train,Sleep Duration
140700,140700,Shivam,Male,53.0,Visakhapatnam,Working Professional,Judge,NaN,2.0,NaN,...,5.0,Moderate,LLB,No,9.0,3.0,Yes,NaN,False,3.830972
140701,140701,Sanya,Female,58.0,Kolkata,Working Professional,Educational Consultant,NaN,2.0,NaN,...,4.0,Moderate,B.Ed,No,6.0,4.0,No,NaN,False,3.145091
140702,140702,Yash,Male,53.0,Jaipur,Working Professional,Teacher,NaN,4.0,NaN,...,1.0,Moderate,B.Arch,Yes,12.0,4.0,No,NaN,False,7.500000
140703,140703,Nalini,Female,23.0,Rajkot,Student,NaN,5.0,NaN,6.84,...,NaN,Moderate,BSc,Yes,10.0,4.0,No,NaN,False,11.757783
140704,140704,Shaurya,Male,47.0,Kalyan,Working Professional,Teacher,NaN,5.0,NaN,...,5.0,Moderate,BCA,Yes,3.0,4.0,No,NaN,False,7.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234495,234495,Zoya,Female,49.0,Jaipur,Working Professional,Pilot,NaN,3.0,NaN,...,5.0,Moderate,BSc,Yes,2.0,2.0,Yes,NaN,False,2.042552
234496,234496,Shlok,Male,29.0,Ahmedabad,Working Professional,Pilot,NaN,5.0,NaN,...,1.0,Moderate,BE,Yes,11.0,3.0,Yes,NaN,False,7.500000
234497,234497,Rishi,Male,24.0,Visakhapatnam,Student,NaN,1.0,NaN,7.51,...,NaN,Moderate,B.Tech,No,7.0,1.0,No,NaN,False,7.500000
234498,234498,Eshita,Female,23.0,Kalyan,Working Professional,Marketing Manager,NaN,4.0,NaN,...,2.0,Healthy,BA,Yes,7.0,5.0,Yes,NaN,False,5.500000


In [349]:
stu_df = pd.DataFrame(df.loc[df["Working Professional or Student"] == "Student"])
stu_df.drop(columns=["id", "Name", "Working Professional or Student", "Profession", "Job Satisfaction", "Work Pressure", "Degree", "Dietary Habits", "City"], inplace=True)
# "Sleep Duration"
# fill the missing values with the mean value of the column
stu_fill_list = ['Academic Pressure', 'CGPA', 'Study Satisfaction']
for col in stu_fill_list:
    stu_df[col] = stu_df[col].fillna(stu_df[col].mean())
stu_df.loc[~stu_df['Train'] == False].dropna(axis=0, inplace=True)
stu_df.reset_index(drop=True, inplace=True)

In [350]:
pro_df = pd.DataFrame(df.loc[df["Working Professional or Student"] == "Working Professional"])
pro_df.drop(columns=["id", "Name", "Working Professional or Student", "Academic Pressure", "CGPA", "Study Satisfaction", "Profession", "Degree", "Dietary Habits", "City"], inplace=True)
# "Sleep Duration"
# fill the missing values with the mean value of the column
pro_fill_list = ['Work Pressure', 'Job Satisfaction']
for col in pro_fill_list:
    pro_df[col] = pro_df[col].fillna(pro_df[col].mean())
# pro_train_data['Profession'] = pro_train_data['Profession'].fillna('Unknown')
pro_df.loc[~pro_df['Train'] == False].dropna(axis=0, inplace=True)
# pro_df.dropna(axis=0, inplace=True)
pro_df.reset_index(drop=True, inplace=True)

In [351]:
# separate labels with features
stu_df_label = stu_df['Depression'].dropna()
stu_df_feature = stu_df.drop(columns='Depression')
pro_df_label = pro_df['Depression'].dropna()
pro_df_feature = pro_df.drop(columns='Depression')

# 獨熱編碼
stu_df_feature = pd.get_dummies(stu_df_feature)
pro_df_feature = pd.get_dummies(pro_df_feature)

In [352]:
stu_train_data_features = stu_df_feature.loc[stu_df_feature['Train'] == True]
pro_train_data_features = pro_df_feature.loc[pro_df_feature['Train'] == True]

In [353]:
# 機器學習 - student
X_train, X_test, y_train, y_test = train_test_split(stu_train_data_features, stu_df_label, test_size=0.2)
stu_model = HistGradientBoostingClassifier() # 83%
stu_model.fit(X_train, y_train)
y_pred = stu_model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.82      0.78      0.80      2342
         1.0       0.85      0.87      0.86      3239

    accuracy                           0.84      5581
   macro avg       0.83      0.83      0.83      5581
weighted avg       0.84      0.84      0.84      5581



In [354]:
X_train, X_test, y_train, y_test = train_test_split(pro_train_data_features, pro_df_label, test_size=0.2)
pro_model = HistGradientBoostingClassifier() # 95%
pro_model.fit(X_train, y_train)
y_pred = pro_model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.97      0.99      0.98     20644
         1.0       0.81      0.68      0.74      1916

    accuracy                           0.96     22560
   macro avg       0.89      0.83      0.86     22560
weighted avg       0.96      0.96      0.96     22560



In [355]:
# stu_test_data = stu_df_feature.loc[stu_df_feature['Train'] == False]
# pro_test_data = pro_df_feature.loc[pro_df_feature['Train'] == False]

# Apply the function to the 'Sleep Duration' column
test_data['Sleep Duration Numeric'] = test_data['Sleep Duration'].apply(parse_sleep_duration)
test_data['Sleep Duration Numeric'].fillna(test_data['Sleep Duration Numeric'].mean(), inplace=True)
test_data.drop(columns='Sleep Duration', inplace=True)
test_data.rename(columns={'Sleep Duration Numeric': 'Sleep Duration'}, inplace=True)

stu_test_data = pd.DataFrame(test_data.loc[test_data["Working Professional or Student"] == "Student"])
pro_test_data = pd.DataFrame(test_data.loc[test_data["Working Professional or Student"] == "Working Professional"])

In [356]:
stu_test_data.drop(columns=["id", "Name", "Working Professional or Student", "Profession", "Job Satisfaction", "Work Pressure", "Degree", "Dietary Habits", "City"], inplace=True)
stu_fill_list = ['Academic Pressure', 'CGPA', 'Study Satisfaction']
for col in stu_fill_list:
    stu_test_data[col] = stu_test_data[col].fillna(stu_test_data[col].mean())

stu_test_data = pd.get_dummies(stu_test_data)

In [357]:
pro_test_data.drop(columns=["id", "Name", "Working Professional or Student", "Academic Pressure", "CGPA", "Study Satisfaction", "City", "Profession", "Degree", "Dietary Habits"], inplace=True)
pro_fill_list = ['Work Pressure', 'Job Satisfaction']
for col in pro_fill_list:
    pro_test_data[col] = pro_test_data[col].fillna(pro_test_data[col].mean())
    
# pro_test_data['Profession'] = pro_train_data['Profession'].fillna('Unknown')
pro_test_data = pd.get_dummies(pro_test_data)

In [358]:
pro_test_data['Depression'] = pro_model.predict(pro_test_data)
stu_test_data['Depression'] = stu_model.predict(stu_test_data)

In [359]:
submission = pd.DataFrame(test_data['id'], columns=['id', 'Depression']) # starting ID 140700
submission['Depression'] = pd.concat([pro_test_data['Depression'], stu_test_data['Depression']]).astype(int).sort_index()

In [360]:
submission.head() # now we need to save as csv

,id,Depression
0,140700,0
1,140701,0
2,140702,0
3,140703,1
4,140704,0


In [361]:
submission.to_csv('submission.csv', index=False, lineterminator='\n') # save the submission